In [1]:
import pandas as pd , os 
from sqlalchemy import create_engine, text as sql_text
from sqlalchemy.orm import Session
from dotenv import load_dotenv
import plotly.express as px

load_dotenv(r'C:\Users\SheharyarMonnoo\myCode\2 - DataPipeLine Automation\Analytics\.env')

True

In [2]:
data = {
    '2023-01': [163864.16, 112937.16, 198976.75, 202631.42, -446.12],
    '2023-02': [171001.34, 105629.91, 180039.57, 184808.94, 402.62],
    '2023-03': [203794.01, 188633.06, 258066.90, 255200.32, 255.73],
    '2023-04': [170459.30, 155811.22, 208092.32, 219693.40, 236.00],
    '2023-05': [138728.68, 129409.28, 175060.89, 176492.31, 3164.99]
}

index = ['Arlington', 'Denton', 'Keller', 'Frisco', 'Unapplied Payment']

df = pd.DataFrame(data, index=index).reset_index()
df.melt('index').groupby(['index'],as_index=False)['value'].sum().to_csv('KareoPmts.csv')

In [3]:
def connectDB():    

    # SQL Server connection parameters
    SERVER = str(os.getenv('SERVER'))
    DATABASE = str(os.getenv('DATABASE'))
    USERNAME_DB = str(os.getenv('USERNAME_DB'))
    PASSWORD_DB = str(os.getenv('PASSWORD_DB'))
    CONNECTION_TYPE = 'pymssql'
    
    connection_str = f'mssql+{CONNECTION_TYPE}://{USERNAME_DB}:{PASSWORD_DB}@{SERVER}/{DATABASE}'
    engine = create_engine(connection_str)
    
    return engine

engine = connectDB()

In [4]:
query = f"""

SELECT *
  FROM [dbo].[NextGenAppts]
  
  where servicePeriod >= '2023-01'
  and servicerPeriod <= '2024-11' 
  and patientName not like '%TEST%'
  and patientName not like '%Peter Pan%'
  and patientName not like '%TRAINING%'


"""

In [5]:
df = pd.read_sql(sql=sql_text(query), con=engine.connect())

servicePeriods = df['servicePeriod'].unique()
servicePeriods.sort()

locations = df['locationName'].unique()
locations

ProgrammingError: (pymssql._pymssql.ProgrammingError) (207, b"Invalid column name 'servicerPeriod'.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n")
[SQL: 

SELECT *
  FROM [dbo].[NextGenAppts]
  
  where servicePeriod >= '2023-01'
  and servicerPeriod <= '2024-11' 
  and patientName not like '%TEST%'
  and patientName not like '%Peter Pan%'
  and patientName not like '%TRAINING%'


]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
data = {
    'Location': ['Arlington', 'Denton', 'Frisco', 'Keller'],
    '2023-01': [567, 516, 691, 804],
    '2023-02': [506, 527, 639, 766],
    '2023-03': [645, 720, 852, 899],
    '2023-04': [568, 638, 771, 822]
}

new_df = pd.DataFrame(data)
# Melt the dataframe to convert from wide format to long format
melted_df = pd.melt(new_df, id_vars=['Location'], var_name='servicePeriod', value_name='Amount')

# Rename columns to match the original dataframe
melted_df = melted_df.rename(columns={'Location': 'locationName'})

In [ ]:
"""Total Encounters"""
sortMatrix = {'Arlington': 1, 'Denton': 2, 'Frisco': 3, 'Keller': 4, 'Weatherford': 5, 'Las Colinas': 6, 'Tyler': 7, 'Sherman':8 }

EncounterCounts = df.groupby(['locationName','servicePeriod'],as_index=False)['Amount'].sum()
# Concatenate with the original dataframe\
EncounterCountsGraph = pd.concat([EncounterCounts, melted_df], ignore_index=True)
# EncounterCountsGraph = EncounterCounts.copy()
EncounterCountsGraph['sortByArray'] = EncounterCountsGraph['locationName'].map(sortMatrix)
EncounterCountsGraph = EncounterCountsGraph.groupby(['locationName','servicePeriod'],as_index=False)['Amount'].sum()
EncounterCountsGraph

,locationName,servicePeriod,Amount
0,Arlington,2023-01,568.0
1,Arlington,2023-02,508.0
2,Arlington,2023-03,645.0
3,Arlington,2023-04,570.0
4,Arlington,2023-05,688.0
...,...,...,...
134,Weatherford,2024-08,738.0
135,Weatherford,2024-09,668.0
136,Weatherford,2024-10,828.0
137,Weatherford,2024-11,719.0


In [ ]:
EncounterCountsGraph.to_clipboard(index=False)

In [ ]:
"""New Patients"""
new_pts = df[(df['appointmentType'].str.lower().str.contains('new')) & (df['servicePeriod'] >= '2023-05')]\
    .groupby(['locationName','servicePeriod','appointmentType'],as_index=False)['patientChartNumber'].count()\
    .rename(columns={'patientChartNumber':'Amount'}).sort_values(by=['locationName','servicePeriod'])
new_pts   

,locationName,servicePeriod,appointmentType,Amount
0,Arlington,2023-05,NEW PATIENT,4
1,Arlington,2023-06,NEW PATIENT,9
2,Arlington,2023-07,NEW PATIENT,7
3,Arlington,2023-08,NEW PATIENT,12
4,Arlington,2023-09,NEW PATIENT,8
...,...,...,...,...
118,Weatherford,2024-08,NEW PATIENT,16
119,Weatherford,2024-09,NEW PATIENT,9
120,Weatherford,2024-10,NEW PATIENT,20
121,Weatherford,2024-11,NEW PATIENT,10


In [ ]:
"""Total Patients"""
total_pts = df[(df['servicePeriod'] >= '2023-05')]\
    .groupby(['servicePeriod','patientChartNumber'],as_index=False)['appointmentType'].count()   

total_pts = total_pts.merge(df, on='patientChartNumber', how='left').drop_duplicates(subset=['patientChartNumber','servicePeriod_x'])\
[['servicePeriod_x','locationName','patientChartNumber']].groupby(['locationName','servicePeriod_x'],as_index=False)['patientChartNumber'].count()\
.rename(columns={'servicePeriod_x':'servicePeriod','patientChartNumber':'Total Patients'})

total_pts

,locationName,servicePeriod,Total Patients
0,Arlington,2023-05,81
1,Arlington,2023-06,87
2,Arlington,2023-07,83
3,Arlington,2023-08,84
4,Arlington,2023-09,79
...,...,...,...
142,Weatherford,2024-08,91
143,Weatherford,2024-09,88
144,Weatherford,2024-10,95
145,Weatherford,2024-11,94


In [ ]:
"""Compliance"""
df['serviceDate'] = pd.to_datetime(df['serviceDate'])

df1 = df[(df['servicePeriod'] >= '2023-05')]\
    .groupby(['locationName','serviceDate','patientChartNumber','servicePeriod'],as_index=False)['appointmentType'].count()

df1['Week'] = df1['serviceDate'].dt.strftime('%U')

df2 = df1.groupby(['locationName','Week','servicePeriod','patientChartNumber'],as_index=False)['appointmentType'].sum()\
    .groupby(['locationName','servicePeriod'],as_index=False)['appointmentType'].mean().round(2) 

df2.head(2)

,locationName,servicePeriod,appointmentType
0,Arlington,2023-05,1.92
1,Arlington,2023-06,1.95


In [ ]:
"""Lost Patients"""

# Find new patients for all periods
all_periods = sorted(df['servicePeriod'].unique())
lost_patients_all_periods = {}

for i in range(1, len(all_periods)):

    current_period = all_periods[i]
    previous_period = all_periods[i - 1]

    # print(f'Current period: {current_period} - Previous period: {previous_period}')

    current_patients = df[df['servicePeriod'] == current_period]['patientChartNumber'].unique()
    previous_patients = df[df['servicePeriod'] == previous_period]['patientChartNumber'].unique()

    lost_patients = set(previous_patients) - set(current_patients)
    

    lost_patients = set(previous_patients) - set(current_patients)
    lost_patients_all_periods[current_period] = lost_patients


In [ ]:
lp_df = pd.DataFrame([
    {'servicePeriod': period, 'patientChartNumber': patient}
    for period, patients in lost_patients_all_periods.items()
    for patient in patients
])

lp_df['count'] = 1

In [ ]:
lp_df = lp_df.groupby(['servicePeriod','patientChartNumber'],as_index=False)['count'].count()\
      .merge(df, on='patientChartNumber', how='left')[['servicePeriod_x','locationName','patientChartNumber','count']]\
      .drop_duplicates(subset=['servicePeriod_x','patientChartNumber'])\
      .groupby(['locationName','servicePeriod_x'],as_index=False)['count'].sum()

In [ ]:
lp_df

,locationName,servicePeriod_x,count
0,Arlington,2023-05,2
1,Arlington,2023-06,7
2,Arlington,2023-07,13
3,Arlington,2023-08,13
4,Arlington,2023-09,15
...,...,...,...
117,Weatherford,2024-08,8
118,Weatherford,2024-09,12
119,Weatherford,2024-10,14
120,Weatherford,2024-11,11


In [ ]:

# Rename columns to have the same name and add a new column to indicate the type
new_pts = new_pts.rename(columns={'appointmentType': 'Type', 'Amount': 'Amount'})
total_pts = total_pts.rename(columns={'Total Patients': 'Amount'})
df2 = df2.rename(columns={'appointmentType': 'Amount'})
lp_df = lp_df.rename(columns={'servicePeriod_x': 'servicePeriod', 'count': 'Amount'})
EncounterCountsGraph = EncounterCountsGraph.rename(columns={'Amount': 'Amount'})

# Add a new column to indicate the type of data
new_pts['Type'] = 'NewPatients'
total_pts['Type'] = 'TotalPatients'
df2['Type'] = 'Compliance'
lp_df['Type'] = 'LostPatients'
EncounterCountsGraph['Type'] = 'Encounters'

# Concatenate the dataframes
combined_df = pd.concat(
    [new_pts[['locationName', 'servicePeriod', 'Amount', 'Type']],
     total_pts[['locationName', 'servicePeriod', 'Amount', 'Type']],
     df2[['locationName', 'servicePeriod', 'Amount', 'Type']],
     lp_df[['locationName', 'servicePeriod', 'Amount', 'Type']],
     EncounterCountsGraph[['locationName', 'servicePeriod', 'Amount', 'Type']]],
    ignore_index=True
)

# combined_df['Quarter'] = pd.to_datetime(combined_df['servicePeriod'] + '-01').dt.to_period('Q').astype(str)
combined_df['Update Time'] = pd.to_datetime('now',utc=True).strftime('%Y-%m-%d %H:%M:%S')
combined_df.head(2)

,locationName,servicePeriod,Amount,Type,Update Time
0,Arlington,2023-05,4.0,NewPatients,2024-12-08 00:11:41
1,Arlington,2023-06,9.0,NewPatients,2024-12-08 00:11:41


In [ ]:
location_updates = {
       'Las Colinas': '2023-12',
       'Weatherford': '2023-10',
       'Sherman': '2024-08',
       'Tyler': '2024-02'
}

for location, period in location_updates.items():
       combined_df.loc[(combined_df['locationName'] == location) & (combined_df['servicePeriod'] < period), 'locationName'] = 'Keller'


In [ ]:
combined_df.head(2)

,locationName,servicePeriod,Amount,Type,Update Time
0,Arlington,2023-05,4.0,NewPatients,2024-12-08 00:11:41
1,Arlington,2023-06,9.0,NewPatients,2024-12-08 00:11:41


In [ ]:
combined_df.to_sql('ScoreCardsMonthly', con=engine, if_exists='replace', index=False)

235